# Seasonal Gas Carry Analysis
This notebook computes the seasonal carry \(winter\–summer\) spread for European gas contracts, evaluates the net storage carry considering fees and losses, plots the spread against breakeven and trigger lines, and runs a simple sensitivity analysis on the winter price. Configuration parameters are read from `config.yaml` and daily settlement data from `gas_seasonal.csv`.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path

# Paths
data_dir = Path('data')
csv_path = data_dir / 'gas_seasonal.csv'
config_path = data_dir / 'config.yaml'

# Load configuration
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)

storage_fee_per_mwh = cfg['storage_fee_per_mwh']
inject_withdraw_cost = cfg['inject_withdraw_cost']
loss_pct = cfg['loss_pct']
threshold_eur_mwh = cfg['threshold_eur_mwh']

# Load data
df = pd.read_csv(csv_path, parse_dates=['date'])
df = df.sort_values('date')
# Compute spread and costs
df['spread'] = df['winter'] - df['summer']
df['cost_dynamic'] = storage_fee_per_mwh + inject_withdraw_cost + (loss_pct * df['winter'])
df['net_pnl'] = df['spread'] - df['cost_dynamic']

# Breakeven and trigger lines (constant using last winter price)
last_winter = df.iloc[-1]['winter']
breakeven = storage_fee_per_mwh + inject_withdraw_cost + (loss_pct * last_winter)
trigger = breakeven + threshold_eur_mwh

# Plot
plt.figure(figsize=(10,5))
plt.plot(df['date'], df['spread'], label='Winter-Summer Spread')
plt.axhline(breakeven, color='red', linestyle='--', label=f'Breakeven ({breakeven:.2f} €/MWh)')
plt.axhline(trigger, color='green', linestyle='--', label=f'Trigger ({trigger:.2f} €/MWh)')
# Annotate latest value
latest = df.iloc[-1]
plt.scatter(latest['date'], latest['spread'], color='black')
plt.text(latest['date'], latest['spread'], f"  Net P&L: {latest['net_pnl']:.2f} €/MWh", verticalalignment='bottom', fontsize=8)
plt.title('Seasonal Gas Spread (Winter - Summer)')
plt.xlabel('Date')
plt.ylabel('Spread (€/MWh)')
plt.legend()
plt.tight_layout()
plt.show()

# Sensitivity analysis on winter price
winter_today = latest['winter']
summer_today = latest['summer']
scenarios = []
for pct in [-0.1, 0.0, 0.1]:
    adj_winter = winter_today * (1 + pct)
    spread_adj = adj_winter - summer_today
    cost_adj = storage_fee_per_mwh + inject_withdraw_cost + (loss_pct * adj_winter)
    net_pnl_adj = spread_adj - cost_adj
    scenarios.append({'scenario': f'{int(pct*100)}% change', 'adjusted_winter_price': adj_winter, 'net_pnl': net_pnl_adj})

sensitivity_df = pd.DataFrame(scenarios)
sensitivity_df


The table above shows the net carry per MWh under baseline conditions and under ±10 % moves in the winter price. A higher winter price increases the spread but also the value lost through storage losses, so the net benefit is not linear. Adjust the configuration in `config.yaml` to explore other storage fees, injection/withdrawal costs or loss percentages.